In [1]:
pip install ntlk


ERROR: Could not find a version that satisfies the requirement ntlk (from versions: none)
ERROR: No matching distribution found for ntlk
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
# IMPORTS
from nltk.corpus import stopwords
import nltk
import pandas as pd
import re
from bs4 import BeautifulSoup
import numpy as np
import gc
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,mean_squared_error
from sklearn import metrics





In [3]:
df = pd.read_csv('IMDB Dataset.csv')


In [5]:
nltk.download('stopwords')
stops = set(stopwords.words('english'))
preprocess_df = pd.DataFrame(columns=['review', 'sentiment'], index=[i for i in range(len(df))])
for i in range(len(df)):
    sentence = df.loc[i,'review']
    text = BeautifulSoup(sentence, "html.parser").get_text()
    pattern=r'[^a-zA-z0-9\s]'                #Remove special characters
    text = re.sub(pattern,'',text)
    text = re.sub('\[[^]]*\]', '', text)     #Remove square brackets
    ps=nltk.porter.PorterStemmer()
    text= ' '.join([ps.stem(word) for word in text.split()])
    wordList = text.split()
    newWordList = []
    for word in wordList:
        # Remove Stop Words
        word = word.lower()
        if word.lower() in stops:
            continue
        else:
            newWordList.append(word)
    text = ' '.join(newWordList)
    preprocess_df.loc[i,'review'] = text
    if df.loc[i,'sentiment'] == 'positive':
        preprocess_df.loc[i,'sentiment'] = "positive"
        df.loc[i,'sentiment'] = "positive"
    else:
        preprocess_df.loc[i,'sentiment'] = "negative"
        df.loc[i,'sentiment'] = "negative"
        
print(preprocess_df.head(5))
preprocess_df = preprocess_df.reset_index(drop = True)
preprocess_df.head(20)
#we drop the duplicate and the remain number of the whole dataset is 49582.

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/shared-libs/python3.9/py-core/lib/python3.9/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
                                              review sentiment
0  one review ha mention watch 1 oz episod youll ...  positive
1  wonder littl product film techniqu veri unassu...  positive
2  thought thi wa wonder way spend time hot summe...  positive
3  basic famili littl boy jake think zombi hi clo...  negative
4  petter mattei love time money visual stun film...  positive


,review,sentiment
0,one review ha mention watch 1 oz episod youll ...,positive
1,wonder littl product film techniqu veri unassu...,positive
2,thought thi wa wonder way spend time hot summe...,positive
3,basic famili littl boy jake think zombi hi clo...,negative
4,petter mattei love time money visual stun film...,positive
5,probabl alltim favorit movi stori selfless sac...,positive
6,sure would like see resurrect date seahunt ser...,positive
7,thi show wa amaz fresh innov idea 70 first air...,negative
8,encourag posit comment thi film wa look forwar...,negative
9,like origin gut wrench laughter like thi movi ...,positive


In [6]:
print(preprocess_df['review'][1])

wonder littl product film techniqu veri unassum veri oldtimebbc fashion give comfort sometim discomfort sens realism entir piec actor extrem well chosen michael sheen onli ha got polari ha voic pat truli see seamless edit guid refer william diari entri onli well worth watch terrificli written perform piec master product one great master comedi hi life realism realli come home littl thing fantasi guard rather use tradit dream techniqu remain solid disappear play knowledg sens particularli scene concern orton halliwel set particularli flat halliwel mural decor everi surfac terribl well done


In [7]:
print(len(preprocess_df['review']))
print(len(df['review']))

50000
50000


In [8]:
nltk.downloader.download('vader_lexicon')
sid = SentimentIntensityAnalyzer()
def get_sentiment(sentence):
    sentiment_score = sid.polarity_scores(sentence)
    #print(sentiment_score)
    return sentiment_score

predicted = []
for index in range(len(preprocess_df)):
    sentiment_score = get_sentiment(preprocess_df['review'][index])
    positive_score = round((sentiment_score['pos'] * 10), 2)
    negative_score = round((sentiment_score['neg'] * 10), 2)
    #print("positive meter = "+str(positive_score))
    #print("negative meter = "+str(negative_score))
    if positive_score < negative_score:
        predicted.append('negative')
    elif positive_score > negative_score:
        predicted.append('positive')
    elif positive_score == negative_score:
        predicted.append('positive')




[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [9]:
df['prediction'] = predicted
df['prediction'].head(10)

0    negative
1    positive
2    positive
3    negative
4    positive
5    positive
6    positive
7    positive
8    negative
9    positive
Name: prediction, dtype: object

In [10]:
df['sentiment'].head(10)

0    positive
1    positive
2    positive
3    negative
4    positive
5    positive
6    positive
7    negative
8    negative
9    positive
Name: sentiment, dtype: object

In [11]:

#confusion_matrix = metrics.confusion_matrix(df['sentiment'], df['prediction'])
#cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [False, True])
#cm_display.plot()
#plt.show()

print(metrics.classification_report(df['sentiment'],df['prediction']))


              precision    recall  f1-score   support

    negative       0.75      0.42      0.54     25000
    positive       0.60      0.86      0.70     25000

    accuracy                           0.64     50000
   macro avg       0.67      0.64      0.62     50000
weighted avg       0.67      0.64      0.62     50000



<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=1830d32f-c8ce-4fc8-8446-4b86a746869d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>